In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import json

# Load MATH dataset (replace 'train.jsonl' with your dataset path)
def load_math_problems(file_path, num_samples=1000):
    problems = []
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            if i >= num_samples:  # Limit to a certain number of samples
                break
            data = json.loads(line)
            problems.append(data["question"])
    return problems

# Load 1000 problems from the dataset
questions = load_math_problems("train.jsonl")

# Tokenize the questions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions)
sequences = tokenizer.texts_to_sequences(questions)
max_len = max(len(seq) for seq in sequences)

# Pad sequences
X = pad_sequences(sequences, maxlen=max_len, padding='post')

# Dummy labels (replace with real labels if available)
y = np.random.rand(len(questions), 1)

# Build LSTM model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=max_len),
    LSTM(64, return_sequences=False),
    Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mse')
model.summary()

# Train the model
model.fit(X, y, epochs=5, verbose=1)  # Reduce epochs for demonstration

# Save the model
model.save('math_question_generator.h5')

print("Model trained and saved successfully.")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - loss: 0.1403
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0822
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0856
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0859
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0846


Model trained and saved successfully.


In [3]:
from flask import Flask, request, jsonify, render_template
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Initialize Flask app
app = Flask(__name__)

# Load pre-trained LSTM model
model = load_model('math_question_generator.h5')

# Math operations for Level 1 (basic arithmetic)
OPERATIONS = ['+', '-', '*', '/']


# Function to generate random questions for each level
def generate_question(level):
    if level == 1:
        # Level 1: Basic arithmetic (fractions, decimals, etc.)
        num1, num2 = random.randint(1, 10), random.randint(1, 10)
        operation = random.choice(OPERATIONS)
        question = f"{num1} {operation} {num2}"
        answer = eval(question)
    elif level == 2:
        # Level 2: Geometry (area and perimeter of simple shapes)
        question = "What is the area of a rectangle with length 5 and width 3?"
        answer = 5 * 3
    elif level == 3:
        # Level 3: Basic algebra
        question = "Solve for x: 2x + 3 = 11"
        answer = (11 - 3) / 2
    elif level == 4:
        # Level 4: Basic calculus
        question = "What is the derivative of 2x^2?"
        answer = "4x"
    else:
        question = "Invalid level"
        answer = None
    return question, answer


# Route to fetch a new question based on the level
@app.route('/get_question', methods=['GET'])
def get_question():
    level = int(request.args.get('level', 1))
    question, answer = generate_question(level)
    return jsonify({'question': question, 'answer': answer})


# Route to check the answer
@app.route('/check_answer', methods=['POST'])
def check_answer():
    data = request.json
    user_answer = float(data['user_answer'])
    correct_answer = float(data['correct_answer'])

    feedback = "Correct!" if np.isclose(user_answer, correct_answer, atol=0.1) else "Incorrect. Try again!"
    return jsonify({'feedback': feedback})


# Route to display progress report
@app.route('/report', methods=['GET'])
def report():
    report_data = {
        "total_questions": 20,
        "correct_answers": 15,
        "accuracy": "75%"
    }
    return jsonify(report_data)


# Run the app
if __name__ == '__main__':
    app.run(debug=True)

TypeError: Could not locate function 'mse'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'keras.metrics', 'class_name': 'function', 'config': 'mse', 'registered_name': 'mse'}